# Практическое задание

In [ ]:
!pip install implicit

     |████████████████████████████████| 71 kB 3.2 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Created wheel for implicit: filename=implicit-0.5.2-cp37-cp37m-linux_x86_64.whl size=22883642 sha256=128e165c3136f7db2c7b379c99628d121cbddbe61662e58472ffcdc8e815bfa9
  Stored in directory: /root/.cache/pip/wheels/3e/2c/6c/dddd8ae30a31d65c3287d074cd9c6f5af31b9819ec56607384
Successfully built implicit


## Предварительные данные

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import bm25_weight, tfidf_weight


In [3]:
data = pd.read_csv('transaction_data.csv')

data.columns = [col.lower() for col in data.columns]
data.rename(columns={'household_key': 'user_id',
                    'product_id': 'item_id'},
           inplace=True)


test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

In [4]:
data

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2595727,1598,42305362535,711,92130,1,0.99,3228,0.00,1520,102,0.0,0.0
2595728,1598,42305362535,711,114102,1,8.89,3228,0.00,1520,102,0.0,0.0
2595729,1598,42305362535,711,133449,1,6.99,3228,0.00,1520,102,0.0,0.0
2595730,1598,42305362535,711,6923644,1,4.50,3228,-0.49,1520,102,0.0,0.0


In [6]:
item_features = pd.read_csv('product.csv')
item_features.columns = [col.lower() for col in item_features.columns]
item_features.rename(columns={'product_id': 'item_id'}, inplace=True)

item_features

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,
2,26093,69,PASTRY,Private,BREAD,BREAD:ITALIAN/FRENCH,
3,26190,69,GROCERY,Private,FRUIT - SHELF STABLE,APPLE SAUCE,50 OZ
4,26355,69,GROCERY,Private,COOKIES/CONES,SPECIALTY COOKIES,14 OZ
...,...,...,...,...,...,...,...
92348,18293142,6384,DRUG GM,National,BOOKSTORE,PAPERBACK BOOKS,
92349,18293439,6393,DRUG GM,National,BOOKSTORE,CHILDRENS LOW END,
92350,18293696,6406,DRUG GM,National,BOOKSTORE,PAPERBACK BEST SELLER,
92351,18294080,6442,DRUG GM,National,BOOKSTORE,PAPERBACK BOOKS,


In [7]:
item_features.department.unique()

array(['GROCERY', 'MISC. TRANS.', 'PASTRY', 'DRUG GM', 'MEAT-PCKGD',
       'SEAFOOD-PCKGD', 'PRODUCE', 'NUTRITION', 'DELI', 'COSMETICS',
       'MEAT', 'FLORAL', 'TRAVEL & LEISUR', 'SEAFOOD', 'MISC SALES TRAN',
       'SALAD BAR', 'KIOSK-GAS', 'ELECT &PLUMBING', 'GRO BAKERY',
       'GM MERCH EXP', 'FROZEN GROCERY', 'COUP/STR & MFG', 'SPIRITS',
       'GARDEN CENTER', 'TOYS', 'CHARITABLE CONT', 'RESTAURANT', 'RX',
       'PROD-WHS SALES', 'MEAT-WHSE', 'DAIRY DELI', 'CHEF SHOPPE', 'HBC',
       'DELI/SNACK BAR', 'PORK', 'AUTOMOTIVE', 'VIDEO RENTAL', ' ',
       'CNTRL/STORE SUP', 'HOUSEWARES', 'POSTAL CENTER', 'PHOTO', 'VIDEO',
       'PHARMACY SUPPLY'], dtype=object)

In [8]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[879517, 934369, 1115576, 1124029, 5572301, 65..."
1,3,"[823704, 834117, 840244, 913785, 917816, 93870..."


In [9]:
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()

In [10]:
# Заведем фиктивный item_id (если юзер покупал товары из топ-5000, то он "купил" такой товар)
data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999

user_item_matrix = pd.pivot_table(data_train, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # Можно пробоват ьдругие варианты
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат saprse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

user_item_matrix.head(3)

C:\Users\edive\anaconda3\lib\site-packages\pandas\core\indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


item_id,202291,397896,420647,480014,545926,707683,731106,818980,819063,819227,...,15926885,15926886,15926887,15926927,15927033,15927403,15927661,15927850,16809471,17105257
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [15]:
%%time

model = AlternatingLeastSquares(factors=100, 
                                regularization=0.001,
                                iterations=15, 
                                calculate_training_loss=True, 
                                num_threads=-1)

model.fit(csr_matrix(user_item_matrix).T.tocsr(),  # На вход item-user matrix
          show_progress=True)

recs = model.recommend(userid=userid_to_id[2],  # userid - id от 0 до N
                        user_items=csr_matrix(user_item_matrix).tocsr(),   # на вход user-item matrix
                        N=5, # кол-во рекомендаций 
                        filter_already_liked_items=False, 
                        filter_items=None, 
                        recalculate_user=True)


Wall time: 3.03 s


In [16]:
[id_to_itemid[rec[0]] for rec in recs]

[1106523, 1133018, 999999, 5569230, 1082185]

In [21]:
def get_recommendations(user, model, N=5):
    res = [id_to_itemid[rec[0]] for rec in 
                    model.recommend(userid=userid_to_id[user], 
                                    user_items=sparse_user_item,   # на вход user-item matrix
                                    N=N, 
                                    filter_already_liked_items=False, 
                                    filter_items=None, 
                                    recalculate_user=True)]
    return res

In [18]:
def precision_at_k(recommended_list, bought_list, k=5):
    
    
    bought_list = bought_list
    recommended_list = recommended_list[:k]    
    flags = np.isin(recommended_list, bought_list)    
    precision = flags.sum() / len(recommended_list)
    
    
    return precision

In [22]:
%%time
    
result['als'] = result['user_id'].apply(lambda x: get_recommendations(x, model=model, N=5))
result.apply(lambda row: precision_at_k(row['als'], row['actual']), axis=1).mean()

Wall time: 34.6 s


0.14927172275238385

In [23]:
result.head(2)

,user_id,actual,als
0,1,"[879517, 934369, 1115576, 1124029, 5572301, 65...","[5569374, 901062, 832678, 1033142, 1005186]"
1,3,"[823704, 834117, 840244, 913785, 917816, 93870...","[1133018, 1106523, 914190, 5569327, 910032]"


In [24]:
model.item_factors.shape

(5001, 100)

In [25]:
model.user_factors.shape

(2500, 100)

In [26]:
fast_recs = model.user_factors @ model.item_factors.T
fast_recs.shape

(2500, 5001)

In [27]:
fast_recs[:10, :10]

array([[-2.79667042e-03,  1.44811809e-01,  1.38888229e-02,
        -4.28365543e-02,  6.49024732e-04,  4.21032310e-03,
         1.07152192e-02,  1.35015443e-01, -1.75051838e-02,
        -3.87488678e-02],
       [-3.48191289e-03,  3.33556533e-02, -2.76512932e-02,
        -1.09890755e-03, -7.59436702e-03, -1.08501576e-02,
        -6.18903153e-03,  3.60683091e-02,  7.68773258e-02,
        -1.19234789e-02],
       [-1.86615549e-02, -2.70767342e-02, -5.44994883e-03,
        -2.87503861e-02, -1.30125266e-02, -1.14255752e-02,
        -2.36011781e-02,  1.06212445e-01,  2.00226575e-01,
         4.25330438e-02],
       [ 1.47151379e-02,  6.06656894e-02,  3.66130136e-02,
         7.14334548e-02,  1.03252511e-02,  1.56613216e-02,
         3.31288506e-03,  1.10480651e-01,  3.04394942e-02,
        -9.15073138e-03],
       [-3.10154865e-03,  1.22420732e-02, -2.75324052e-03,
         3.36451605e-02,  2.49803532e-03,  1.95615529e-03,
        -3.91499931e-03,  3.73681933e-02,  4.31690849e-02,
         2.

## Production

Начиная с этого вебинара, мы будем строить *базовое решение* для системы рекомендаций топ-N товаров. В финальном проекте вам нужно будет его существенно улучшить.  
  
**Ситуация**: Вы работает data scientist в крупном продуктовом российском ритейлере. Ваш конкурент сделал рекомендательную систему, и его продажи выросли. Ваш менеджмент тоже хочет увеличить продажи   
**Задача со слов менеджера**: Сделайте рекомендательную систему топ-10 товаров для рассылки по e-mail

**Ожидание:**
- Отправляем e-mail с топ-10 товарами, отсортированными по вероятности

**Реальность:**
- Чего хочет менеджер от рекомендательной системы? (рост показателя X на Y% за Z недель)
- По-хорошему надо бы предварительно посчитать потенциальный эффект от рекоммендательной системы (Оценки эффектов у менеджера и у вас могут сильно не совпадать: как правило, вы знаете про данные больше)
- А у нас вообще есть e-mail-ы пользователей? Для скольки %? Не устарели ли они?
- Будем ли использовать СМС и push-уведомления в приложении? Может, будем печатать рекомендации на чеке после оплаты на кассе?
- Как будет выглядеть e-mail? (решаем задачу топ-10 рекомендаций или ранжирования? И топ-10 ли?)
- Какие товары должны быть в e-mail? Есть ли какие-то ограничения (только акции и т п)?
- Сколько денег мы готовы потратить на привлечение 1 юзера? CAC - Customer Aquisition Cost. Обычно CAC = расходы на коммуникацию + расходы на скидки
- Cколько мы хотим зарабатывать с одного привлеченного юзера?
---
- А точно нужно сортировать по вероятности? (наверняка необязательно)
- Какую метрику использовать? (можно попробовать использовать косинусную близость с implicit ratings, если рекомендации персонализированные. Или другую "похожесть" на ранее купленные товары".)
- Сколько раз в неделю отпрпавляем рассылку? (раз в неделю, не чаще. Ежедневные рассылки чаще всего отправляются в спам или просто пролистываются не привлекая внимания)
- В какое время отправляем рассылку? (15:00 — 17:00. В будние дни это время лучше всего подходит для рекламной рассылки. Люди в предвкушении окончания рабочего дня с удовольствием читают любые рекламные письма, а также активно переходят по ссылкам. А в пятницу в это время отсылать письма рискованно. 9:00 — 10:00. Второе по популярности время для почтовой рассылки. В это время люди не только внимательно читают письма любой тематики, но и активно обмениваются интересными ссылками с коллегами. 19:00 — 22:00. Это время интернет-магазинов. Именно в это время люди чаще всего совершают покупки после прочтения рекламного письма. Также в это время люди, как правило, отвечают на письма.)
- Будем отправлять одному юзеру много раз наши рекоммендации. Как добиться того, чтобы они хоть немного отличались? (переобучать модель периодически. Для каждого юзера делать список, например, из 50 рекомендаций, а в рассылку включать только 5-10, часть из них из топа, часть рандомно из рекомендованного списка)
- Нужно ли, чтобы в одной рассылке были *разные* товары? Как определить, что товары *разные*? Как добиться того, чтобы они были разными?
- И многое другое:)

**В итоге договорились, что:**
- Хотим повысить выручку минимум на 6% за 4 месяца. Будем повышать за счет роста Retention минимум на  3% и среднего чека минимум на 3%
- Топ-5 товаров, а не топ-10 (В e-mail 10 выглядят не красиво, в push и на чек больше 5 не влезает)
- Рассылаем в e-mail (5% клиентов) и push-уведомлении (20% клиентов), печатаем на чеке (все оффлайн клиенты)
- **3 товара с акцией** (Как это учесть? А если на товар была акция 10%, а потом 50%, что будет стоять в user-item матрице?. Вероятно нужно создать некую товарную матрицу, в которой учитывать акции, категории товара и т.д.)
- **1 новый товар** (юзер никогда не покупал. Просто фильтруем аутпут ALS? А если у таких товаров очень маленькая вероятность покупки? Может, использовать другую логику/модель? можно рекомендовать товар из списка покупак "похожих" юзеров. К примеру если про кино - целевой юзер любитель боевиков, другие любители боевиков высоко оценили фильм N, а целевой юзер его не смотрел, стоит ему это порекомендовать "на основе его предпочтений") 
- **1 товар для роста среднего чека** (товары минимум дороже чем обычно покупает юзер. Как это измерить? На сколько дороже? Сделать статистику по средним затратам на категорию товара, предлагать товары из этой категории дороже полученной цифры. Дороже процентов на 10-20, или исходя из поставленной задачи, товары вдвое дороже обычной покупки вряд ли заинтересуют кого-то)

### **Теоретические предположения**

**3 товара с акцией** 

Если нужно, чтобы в топ-5 наиболее подходящих товаров попало три товара с акцией, то или организуется акция на какие-то из этих пяти, или берем первые топ-N наиболее подходящих, в которые попадают первые три акционных, на данный момент товара. Соответственно, получаем 3 товара с акцией и два первых неакционных товара из топ-5. Акционные товары имеют дополнительно весовую характеристику и соответственно, если меняется процент, то меняется и характеристика, что сказывается на дальнейшем выборе подходящих товаров.

**1 новый товар**

Товар должен быть рекомендован из товаров с наибоее близкими показателями (например, косинусное сходство) к его потребительской корзине (возможно новый сорт или новый вид/подвид какого-либо продукта с новыми качествами). Также можно сделать этот товар с акцией, тем самым мотивировать на покупку (переплетение с предыдущим пунктом - 3 товара с акцией)

**1 товар для роста среднего чека**

Средний чек - это соотношение суммы покупок к количеству заказов (средний доход с чека). Можно добавить новый товар со скидкой (переплетение с предыдущим пунктом - 1 новый товар), но который всё равно будет добавлять стоимости его корзине. Как было сказано в первом пункте - минимум на 3%.

### **Практические действия**

Вопросов стало еще больше. Поэтому сначала делаем **MVP** (Minimum viable product) на e-mail. Показываем его менеджеру, измеряем метрики на юзерах. По фидбеку и метрикам делаем улучшения MVP и раскатываем его на push-уведомления и чеки

*Data Science проект* - итеративный процесс!

In [28]:
merged_data = pd.merge(data, item_features, how='left')


In [29]:
merged_data.head(3)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0,69,PRODUCE,Private,POTATOES,POTATOES RUSSET (BULK&BAG),5 LB
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0,2,PRODUCE,National,ONIONS,ONIONS SWEET (BULK&BAG),40 LB
2,2375,26984851472,1,1036325,1,0.99,364,-0.3,1631,1,0.0,0.0,69,PRODUCE,Private,VEGETABLES - ALL OTHERS,CELERY,


In [ ]:
def prefilter_items(data):

    data['price'] = data['sales_value'] / (np.maximum(data['quantity'], 1))

    # Самые популярные товары
    popularity = data.groupby('item_id')['user_id'].nunique().reset_index() 
    popularity['share_unique_users'] = popularity['user_id'] / data['user_id'].nunique()
    
    # Уберем самые популярные товары
    top_popular = popularity[popularity['share_unique_users'] > 0.005].item_id.tolist()
    popul = data[~data['item_id'].isin(top_popular)]

    # Уберем самые НЕ популярные товары (их и так НЕ купят)
    top_notpopular = popularity[popularity['share_unique_users'] < 0.0005].item_id.tolist()
    notpopul = popul[~popul['item_id'].isin(top_notpopular)]

    # Уберем не интересные для рекоммендаций категории (department). Оставим первые 10 категорий
    categories = data.groupby('department').count().reset_index().sort_values('item_id', ascending = False)[:10]
    categories = categories.department.tolist()
    res = notpopul[~notpopul['item_id'].isin(categories)]
    print(res.shape)

    # Уберем товары, которые не продавались за последние 12 месяцев
    res.drop(res[res.week_no > 52].index, inplace=True)

    # Уберем дешевые товары Покупки меньше 1$. 
    res.drop(res[res['price'] < 1].index, inplace=True)

    # Уберем дорогие товары. Покупки больше 100$.
    res.drop(res[res['price'] > 100].index, inplace=True)

    return res

   
def postfilter_items(user_id, recommednations):
    pass

In [30]:
from sklearn.model_selection import GridSearchCV

In [32]:
parameters = {'factors': [100], 'regularization': [0.001, 0.01, 0.0001], 'iterations': [10, 15, 20]}

In [44]:
recommended_list = AlternatingLeastSquares.recommend(userid=userid_to_id[56], 
                                    user_items=sparse_user_item,
                                    N=5,
                                    filter_already_liked_items=False, 
                                    filter_items=None, 
                                    recalculate_user=False)

TypeError: recommend() takes at least 3 positional arguments (0 given)

In [36]:
GS = GridSearchCV(AlternatingLeastSquares, parameters, scoring=precision_at_k(recommended_list, bought_list, k=5), n_jobs=-1)

NameError: name 'recommended_list' is not defined

In [34]:
GS.fit(csr_matrix(user_item_matrix).T.tocsr())

TypeError: If no scoring is specified, the estimator passed should have a 'score' method. The estimator <class 'implicit.als.AlternatingLeastSquares'> does not.

Все эти функции отправим затем в *utils.py*

**ДЗ**

1) Попытаться ответить на вопросы/выдвинуть гипотезы

2) Доделать прошлые домашния задания

3) Прочитать статьи BM25/MatrixFactorization

4) Поэкспериментировать с ALS (grid-search)

**Links**

BM25

https://en.wikipedia.org/wiki/Okapi_BM25#:~:text=BM25%20is%20a%20bag%2Dof,slightly%20different%20components%20and%20parameters.


Matrix factorization (ALS, SVD)
https://datasciencemadesimpler.wordpress.com/tag/alternating-least-squares/